# Smart Facility: Robotics Basic Dataset - Demo Notebook

Demo 1: Organizing the dataset's metadata into a dataframe.  
Demo 2: Visualizing labels for a specific video.

Note: this notebook relies on the `infinity-tools` python module, which can be installed following instructions [here](https://github.com/toinfinityai/infinity-tools#installation).

In [1]:
import os
import glob
import json
import shutil
import pandas as pd
from pycocotools.coco import COCO
from IPython.display import clear_output
from infinity_tools.visionfit import vis
from infinity_tools.common.vis.videos import parse_video_frames, stack_videos

## Demo 1: Dataset Filtering

We first show how to load the dataset's metadata into an easily filtered dataframe.

In [2]:
PATH_TO_DATASET_FOLDER = "PATH_TO_DATASET_FOLDER"

In [3]:
all_param_paths = sorted(glob.glob(os.path.join(PATH_TO_DATASET_FOLDER, "*_params.json")))

metadata = []
for param_path in all_param_paths:
    metadata_dict = {}
    metadata_dict["video_number"] = os.path.basename(param_path).split("_")[0]
    parameters = json.load(open(param_path))["params"]
    metadata_dict.update(parameters)
    metadata.append(metadata_dict)

metadata = pd.DataFrame(metadata)
metadata.head()

,video_number,scene,num_people,frame_rate,random_seed,num_cameras,camera_type,camera_focal_length,camera_field_of_view,camera0_height,...,camera1_height,camera1_yaw_deg,camera_pitch_deg,camera_min_path_length,image_width,image_height,enable_clutter,min_dist_between_clutter_objects,video_duration,camera_speed
0,00000,WAREHOUSE_0001,5,12,617,2,PERSPECTIVE,32.0,82.0,0.3,...,0.3,180.0,0.0,5.0,1024,768,True,3.0,10.706275,1.937161
1,00001,WAREHOUSE_0001,5,12,611,2,PERSPECTIVE,32.0,82.0,0.3,...,0.3,180.0,0.0,5.0,1024,768,True,3.0,10.051719,1.554835
2,00002,WAREHOUSE_0001,5,12,607,2,PERSPECTIVE,32.0,82.0,0.3,...,0.3,180.0,0.0,5.0,1024,768,True,3.0,10.860921,0.916711
3,00003,WAREHOUSE_0001,5,12,637,2,PERSPECTIVE,32.0,82.0,0.3,...,0.3,180.0,0.0,5.0,1024,768,True,3.0,14.680993,1.637403
4,00004,WAREHOUSE_0001,5,12,625,2,PERSPECTIVE,32.0,82.0,0.3,...,0.3,180.0,0.0,5.0,1024,768,True,3.0,12.343555,1.234078


You can now filter and/or query the dataset for specific properties, as shown in the example below.

In [4]:
filtered_samples = metadata.query('camera_speed < 1')
display(filtered_samples.head())

,video_number,scene,num_people,frame_rate,random_seed,num_cameras,camera_type,camera_focal_length,camera_field_of_view,camera0_height,...,camera1_height,camera1_yaw_deg,camera_pitch_deg,camera_min_path_length,image_width,image_height,enable_clutter,min_dist_between_clutter_objects,video_duration,camera_speed
2,00002,WAREHOUSE_0001,5,12,607,2,PERSPECTIVE,32.0,82.0,0.3,...,0.3,180.0,0.0,5.0,1024,768,True,3.0,10.860921,0.916711
5,00005,WAREHOUSE_0001,5,12,621,2,PERSPECTIVE,32.0,82.0,0.3,...,0.3,180.0,0.0,5.0,1024,768,True,3.0,13.523961,0.662400


## Demo 2: Label Visualization

We next show how to visualize labels for a specific video in the robotics dataset, including instance segmentation masks, boundings boxes, 2D keypoints, and cuboids. We do this using the dataset sample provided in the repo.

In [5]:
def visualize_all_labels(video_rgb_path: str) -> str:
    """Visualizes RGB, bounding boxes and segmentation masks in single video."""

    output_directory = os.path.dirname(video_rgb_path)
    labels_path = video_rgb_path.replace(".mp4", "_labels.json")
    seg_zip_path = video_rgb_path.replace(".mp4", "_segmentation.zip")
    
    seg_extracted_path = seg_zip_path.replace(".zip", "")
    os.makedirs(seg_extracted_path, exist_ok=True)
    vis.unzip(seg_zip_path, seg_extracted_path)

    fps = 12
    imgs = parse_video_frames(video_rgb_path)
    coco = COCO(labels_path)

    bounding_box_path = vis.create_bounding_boxes_video(
        os.path.join(output_directory, "labels_bounding_box.mp4"), 
        imgs, 
        fps, 
        coco,
        add_text=True,
    )
    
    segmentation_path = vis.create_segmentation_video(
        os.path.join(output_directory, "labels_segmentation.mp4"),
        seg_extracted_path,
        fps,
        imgs.shape[2],
        imgs.shape[1]
    )
    
    skeleton_path = vis.create_keypoint_connections_video(
        os.path.join(output_directory, "labels_skeleton.mp4"), 
        imgs, 
        fps, 
        coco
    )

    cuboid_path = vis.create_cuboids_video(
        os.path.join(output_directory, "labels_cuboid.mp4"), 
        imgs, 
        fps, 
        coco
    )
        
    shutil.rmtree(seg_extracted_path)
    clear_output()
    
    print(f"The resulting label videos can be viewed at:")
    for e in [bounding_box_path, segmentation_path, skeleton_path, cuboid_path]:
        print(f"\t{e}")

In [6]:
video_rgb_path = "./sample_data/00005_cam0.mp4"
labels_video_path = visualize_all_labels(video_rgb_path)

The resulting label videos can be viewed at:
	./sample_data/labels_bounding_box.mp4
	./sample_data/labels_segmentation.mp4
	./sample_data/labels_skeleton.mp4
	./sample_data/labels_cuboid.mp4
